In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [ ]:
train_data.info()
test_data.info()

In [ ]:
# train_X = train_data.drop('label')            drop 默认删除 行
train_X = train_data.drop('label', axis = 1)  # 添加 axis 将删除列

# train_Y = train_data['label']  使用这种方式 得到的是 一维数组，为（42000，）
train_Y = train_data[['label']] #使用这种方式 得到的是 二维数组 为（42000,1）

In [ ]:
print("train_X: ", train_X.shape, "\ntrain_Y: ", train_Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_Y, test_size=0.2)

In [ ]:
print("X_train: ", X_train.shape, "     Y_train: ", Y_train.shape)
print("X_test: ", X_test.shape, "       Y_test: ", Y_test.shape)

# ResNet

In [ ]:
# X_train = X_train.values
X_train_R = X_train.values.reshape([-1, 28, 28, 1])

# X_test = X_test.values
X_test_R = X_test.values.reshape([-1, 28, 28, 1])

In [ ]:
X_train_R.shape

In [ ]:
from __future__ import division, print_function, absolute_import
import tflearn
import tflearn.data_utils as du

In [ ]:
X_train_R, mean = du.featurewise_zero_center(X_train_R)

In [ ]:
X_test_R = du.featurewise_zero_center(X_test_R, mean)

In [ ]:
net = tflearn.input_data(shape=[None, 28, 28, 1])
net = tflearn.conv_2d(net, 64, 3, activation='relu', bias=False)

In [ ]:
net = tflearn.residual_bottleneck(net, 3, 16, 64)
net = tflearn.residual_bottleneck(net, 1, 32, 128, downsample=True)
net = tflearn.residual_bottleneck(net, 2, 32, 128)
net = tflearn.residual_bottleneck(net, 1, 64, 256, downsample=True)
net = tflearn.residual_bottleneck(net, 2, 64, 256)
net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
net = tflearn.global_avg_pool(net)
net = tflearn.fully_connected(net, 10, activation='softmax')

In [ ]:
net = tflearn.regression(net, optimizer='momentum', loss='categorical_crossentropy', learning_rate=0.1)
model = tflearn.DNN(net, checkpoint_path='model_Digit', max_checkpoints=2, tensorboard_verbose=0,tensorboard_dir='logs')
model.fit(X_train_R, Y_train, n_epoch=1, validation_set=(X_test_R, Y_test), show_metric=True, batch_size=256,run_id='Digit')